## ------------------------------------------------------------------------------------------------
# Face Recognition
## ------------------------------------------------------------------------------------------------

In [ ]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
import os
import threading

#Custom module
import mail
import WhatsApp
import AWS

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

## ------------------------------------------------------------------------------------------------
# Load the model of person1 and person2 
## ------------------------------------------------------------------------------------------------

In [ ]:
person1_saved_model = cv2.face_LBPHFaceRecognizer.create()
person2_saved_model = cv2.face_LBPHFaceRecognizer.create()

person1_saved_model.read('./saved_model/person1_model.yml')
print("\n-------------------------------------------------------------------------")
print("\n Person1 trained model is loaded successfully.")
print("\n-------------------------------------------------------------------------\n")

person2_saved_model.read('./saved_model/person2_model.yml')
print("\n-------------------------------------------------------------------------")
print("\n Person2 trained model is loaded successfully.")
print("\n-------------------------------------------------------------------------\n\n")

## ------------------------------------------------------------------------------------------------

# Recognize person1 face :

## 1. Send Mail with the image of the person1.
## 2. Send WhatsApp message to the given number.

## ------------------------------------------------------------------------------------------------
## ------------------------------------------------------------------------------------------------


# Recognize person2 face :

## Orchestrate EC2 instance and EBS volume (& attach both) on AWS using Terraform.

## ------------------------------------------------------------------------------------------------

In [ ]:
def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
     
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi


# Open Webcam
cap = cv2.VideoCapture(0)

flag1=False
flag2=False

while True:
    ret, frame = cap.read()
    image1, face1 = face_detector(frame)
    image2, face2 = face_detector(frame)
    
    try:
        face1 = cv2.cvtColor(face1, cv2.COLOR_BGR2GRAY)
        face2 = cv2.cvtColor(face2, cv2.COLOR_BGR2GRAY)
        
        # Pass face to prediction model
        # "results" comprises of a tuple containing the label and the confidence value
        result1 = person1_saved_model.predict(face1)
        result2 = person2_saved_model.predict(face2)
        # harry_model.predict(face)
        
        #person1
        #def person1_r():
        if result1[1] < 500:
            confidence1 = int( 100 * (1 - (result1[1])/400) )
            if confidence1 > 89:
                display_string = str(confidence1) + '% Confident of Rahul'
                cv2.putText(image1, display_string, (50, 60), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2) #50 60

        #person2
        #def person2_s():
        if result2[1] < 500:
            confidence2 = int( 100 * (1 - (result2[1])/400) )
            if confidence2 > 89:
                display_string = str(confidence2) + '% Confident of Sneha'  
                cv2.putText(image2, display_string, (80, 90), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2) #100 120
   
        #t1=threading.Thread(target=person1_r)
        #t1.start()
        #t2=threading.Thread(target=person2_s)
        #t2.start()

    
        #person1
        if confidence1 > 89 and flag1 == False:
            #cv2.putText(image1, "Hey Buddy", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image1 )
            print("-------------------------------------------------------------------------\n")
            print("\n --> Rahul's face Recognized.")
            print("-------------------------------------------------------------------------\n")
            mail.securityMail()
            print("\n 1. Mail send with the person1 image.")
            WhatsApp.message()
            print("\n 2. WhatsApp message sent to the given phone number.\n")
            flag1 =True
            
        #person2
        if confidence2 > 89 and flag2 == False:
            #cv2.putText(image2, "Hey Buddy", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image2 )
            print("-------------------------------------------------------------------------")
            print("-------------------------------------------------------------------------\n")
            print("\n --> Sneha's face Recognized.")
            print("\n-------------------------------------------------------------------------\n")
            AWS.ec2_va()
            print("\n 1. Orchestration on AWS using Terraform Started.")
            print("\n-------------------------------------------------------------------------\n")
            flag2 =True        
        
       
        else:
            #person1
            if confidence1 > 89:
                cv2.putText(image1, "R_True", (200, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0), 2)
                cv2.imshow('Face Recognition', image1 )
            #person2
            if confidence2 > 89:
                cv2.putText(image2, "S_True", (350, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0), 2)
                cv2.imshow('Face Recognition', image2 )

    except:
        cv2.putText(image1, "No Face Found", (200, 100) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image1, "looking for face", (300, 500), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image1 )
        #person2
        cv2.putText(image2, "No Face Found", (200, 100) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image2, "looking for face", (300, 500), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image2 )
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
        
cap.release()
cv2.destroyAllWindows() 

print("\n Done!")
print("\n-------------------------------------------------------------------------\n")